In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly as py
import datetime
import seaborn as sns
from google.colab import drive
from sklearn import preprocessing
py.offline.init_notebook_mode(connected=True)
import tensorflow as tf
from tensorflow import keras
from keras.layers import Flatten
from keras.layers import Dense
from keras.models import Sequential
from keras.layers import GlobalAveragePooling1D
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import LeakyReLU
from keras import optimizers
from keras.layers import Dense, Activation, Dropout
from keras.layers import AveragePooling1D
from keras.layers import TimeDistributed
from keras.layers import CuDNNLSTM
from keras.layers import LSTM
from keras.layers import Reshape
from sklearn import preprocessing
from sklearn.metrics import precision_recall_fscore_support
from matplotlib.pyplot import figure
from keras.callbacks import ModelCheckpoint
from keras.backend import clear_session
import gc

Using TensorFlow backend.


In [0]:



drive.mount('/content/drive/')
aapl = pd.read_csv("/content/drive/My Drive/Stock Price Prediction Research/Data/aapl_prices_fiveminute_stats_wsh.csv")
aapl.head(5)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive/


,Unnamed: 0,PRICE_MEAN,VOLUME_MEAN,PRICE_HIGH,PRICE_LOW,PRICE_OPEN,PRICE_CLOSE
0,1/2/2001 9:30,14.930692,565.354331,15.0000,14.8750,14.8750,14.8750
1,1/2/2001 9:35,14.936993,707.792208,15.0625,14.6875,14.8750,14.9375
2,1/2/2001 9:40,15.021382,944.939271,15.0625,14.9375,14.9375,15.0625
3,1/2/2001 9:45,15.114291,735.205992,15.2500,15.0000,15.0625,15.0625
4,1/2/2001 9:50,14.938517,1075.739645,15.0625,14.8125,15.0625,14.9375


In [0]:
date_time = pd.to_datetime(aapl['Unnamed: 0'], infer_datetime_format=True)
datetime_table = pd.DataFrame({'DAY':date_time.dt.day, 'MONTH':date_time.dt.month,'YEAR':date_time.dt.year,'HOUR': date_time.dt.hour, 'MINUTE': date_time.dt.minute})
aapl = aapl.join(datetime_table, how = 'outer')

In [0]:
aapl

,Unnamed: 0,PRICE_MEAN,VOLUME_MEAN,PRICE_HIGH,PRICE_LOW,PRICE_OPEN,PRICE_CLOSE,DAY,MONTH,YEAR,HOUR,MINUTE
0,1/2/2001 9:30,14.930692,565.354331,15.00000,14.8750,14.87500,14.87500,2,1,2001,9,30
1,1/2/2001 9:35,14.936993,707.792208,15.06250,14.6875,14.87500,14.93750,2,1,2001,9,35
2,1/2/2001 9:40,15.021382,944.939271,15.06250,14.9375,14.93750,15.06250,2,1,2001,9,40
3,1/2/2001 9:45,15.114291,735.205992,15.25000,15.0000,15.06250,15.06250,2,1,2001,9,45
4,1/2/2001 9:50,14.938517,1075.739645,15.06250,14.8125,15.06250,14.93750,2,1,2001,9,50
5,1/2/2001 9:55,14.834146,476.146789,14.93750,14.7500,14.87500,14.81250,2,1,2001,9,55
6,1/2/2001 10:00,14.823621,776.156584,15.00000,14.7500,14.75000,15.00000,2,1,2001,10,0
7,1/2/2001 10:05,14.724609,682.812500,14.81250,14.6875,14.81250,14.68750,2,1,2001,10,5
8,1/2/2001 10:10,14.759727,1184.105960,14.81250,14.6250,14.75000,14.81250,2,1,2001,10,10
9,1/2/2001 10:15,14.809375,430.000000,14.87500,14.7500,14.78125,14.81250,2,1,2001,10,15


In [0]:
def stocksplit(stock,day, month, year, ratio):
  date_index = stock.loc[(stock['YEAR'] == year) & (stock['DAY'] == day) & (stock['MONTH']==month)].index.values.astype(int)[0]
  pre_split_high  = stock.iloc[:date_index]['PRICE_HIGH']
  pre_split_low = stock.iloc[:date_index]['PRICE_LOW']
  pre_split_open = stock.iloc[:date_index]['PRICE_OPEN']
  pre_split_close = stock.iloc[:date_index]['PRICE_CLOSE']
  pre_split_mean = stock.iloc[:date_index]['PRICE_MEAN']
  pre_split_volume = stock.iloc[:date_index]['VOLUME_MEAN']

  print(pre_split_volume)
  post_split_high  = stock.iloc[date_index:]['PRICE_HIGH']
  post_split_low = stock.iloc[date_index:]['PRICE_LOW']
  post_split_open = stock.iloc[date_index:]['PRICE_OPEN']
  post_split_close = stock.iloc[date_index:]['PRICE_CLOSE']
  post_split_mean = stock.iloc[date_index:]['PRICE_MEAN']
  post_split_volume = stock.iloc[date_index:]['VOLUME_MEAN']
  print(post_split_volume)
  post_split_high  = post_split_high*2
  post_split_low = post_split_low*2
  post_split_open = post_split_open*2
  post_split_close = post_split_close*2
  post_split_mean = post_split_mean*2
  post_split_volume = post_split_volume/2
  
  print(post_split_volume)
  
  price_high  = pre_split_high.append(post_split_high)
  price_low = pre_split_low.append(post_split_low)
  price_open = pre_split_open.append(post_split_open)
  price_close = pre_split_close.append(post_split_close)
  price_mean = pre_split_mean.append(post_split_mean)
  volume = pre_split_volume.append(post_split_volume)
  stock["PRICE_HIGH"] = price_high
  stock["PRICE_LOW"] = price_low
  stock["PRICE_OPEN"] = price_open
  stock["PRICE_CLOSE"] = price_close
  stock["PRICE_MEAN"] = price_mean
  stock["VOLUME_MEAN"] = volume
  return stock

In [0]:
aapl = stocksplit(aapl, 28, 2, 2005, 2)
aapl

0         565.354331
1         707.792208
2         944.939271
3         735.205992
4        1075.739645
5         476.146789
6         776.156584
7         682.812500
8        1184.105960
9         430.000000
10        659.259259
11        777.941176
12        987.500000
13        891.304348
14        791.891892
15        817.894737
16        737.777778
17        413.793103
18        746.601942
19       1081.896552
20       1490.000000
21        750.515464
22        639.325843
23       1032.692308
24        856.862745
25       1876.543210
26        736.585366
27       1118.367347
28        505.000000
29        527.586207
            ...     
81947     256.159558
81948     214.733796
81949     253.118367
81950     228.141463
81951     217.959322
81952     325.670068
81953     306.020690
81954     211.221344
81955     255.937639
81956     274.786448
81957     282.204852
81958     261.264463
81959     283.935091
81960     261.212377
81961     246.748654
81962     207.331190
81963     193

,Unnamed: 0,PRICE_MEAN,VOLUME_MEAN,PRICE_HIGH,PRICE_LOW,PRICE_OPEN,PRICE_CLOSE,DAY,MONTH,YEAR,HOUR,MINUTE
0,1/2/2001 9:30,14.930692,565.354331,15.00000,14.8750,14.87500,14.87500,2,1,2001,9,30
1,1/2/2001 9:35,14.936993,707.792208,15.06250,14.6875,14.87500,14.93750,2,1,2001,9,35
2,1/2/2001 9:40,15.021382,944.939271,15.06250,14.9375,14.93750,15.06250,2,1,2001,9,40
3,1/2/2001 9:45,15.114291,735.205992,15.25000,15.0000,15.06250,15.06250,2,1,2001,9,45
4,1/2/2001 9:50,14.938517,1075.739645,15.06250,14.8125,15.06250,14.93750,2,1,2001,9,50
5,1/2/2001 9:55,14.834146,476.146789,14.93750,14.7500,14.87500,14.81250,2,1,2001,9,55
6,1/2/2001 10:00,14.823621,776.156584,15.00000,14.7500,14.75000,15.00000,2,1,2001,10,0
7,1/2/2001 10:05,14.724609,682.812500,14.81250,14.6875,14.81250,14.68750,2,1,2001,10,5
8,1/2/2001 10:10,14.759727,1184.105960,14.81250,14.6250,14.75000,14.81250,2,1,2001,10,10
9,1/2/2001 10:15,14.809375,430.000000,14.87500,14.7500,14.78125,14.81250,2,1,2001,10,15


# **FEATURE** **ENGINEERING**


In [0]:
def diff(df, col):
  orig_df = df.copy()
  difference = pd.DataFrame({'DIFF_'+col:np.diff(df[col])})
  orig_df = orig_df.join(difference, how = 'outer')
  return orig_df
  

In [0]:
aapl_diff = diff(aapl, 'PRICE_OPEN')
aapl_diff = diff(aapl_diff, 'PRICE_CLOSE')
aapl_diff = diff(aapl_diff, 'PRICE_HIGH')
aapl_diff = diff(aapl_diff, 'PRICE_LOW')
aapl_diff = diff(aapl_diff, 'PRICE_MEAN')
aapl_diff = diff(aapl_diff, 'VOLUME_MEAN')



In [0]:
def perc_change(df, col):
  original = df.loc[:,col]
  difference = df.loc[:, 'DIFF_'+ col]
  percentage = difference/original
  perc_df = pd.DataFrame({'PERC_'+col: percentage})
  df = df.join(perc_df, how = 'outer')
  return df


In [0]:
aapl_diff = perc_change(aapl_diff, 'PRICE_OPEN')
aapl_diff = perc_change(aapl_diff, 'PRICE_CLOSE')

In [0]:
def range_func(df):
  high = df.PRICE_HIGH
  low = df.PRICE_LOW
  hl_range = high-low 
  difference = np.diff(hl_range)
  diff = pd.DataFrame({'RANGE':difference})
  df=df.join(diff, how= 'outer')
  return df

In [0]:
aapl_diff = range_func(aapl_diff)

In [0]:
def time_of_day(df):
  orig_df = df.copy()
  hour = orig_df.HOUR 
  part_of_day = [1 if x < 11 else 2 if x<13 else 3 for x in hour]
  part_of_day = pd.DataFrame({'TIME':part_of_day})
  orig_df = orig_df.join(part_of_day,how = 'outer')
  return orig_df

In [0]:
aapl_diff = time_of_day(aapl_diff)

In [0]:
time_of_day = pd.get_dummies(aapl_diff.TIME)
time_of_day = time_of_day.rename(columns={1:'MORNING',2:'AFTERNOON', 3:'EVENING'})
aapl_diff = aapl_diff.join(time_of_day, how = 'outer')

In [0]:
def day_of_week(df):
  orig_df = df.copy()
  date = [datetime.date(y,m,d) for y,m,d in zip(orig_df.YEAR, orig_df.MONTH, orig_df.DAY)]
  part_of_week = [x.weekday() for x in date]
  part_of_week = pd.DataFrame({'DAY_OF_WEEK': part_of_week})
  orig_df = orig_df.join(part_of_week, how = 'outer')
  return orig_df

In [0]:
aapl_diff = day_of_week(aapl_diff)

In [0]:
day_of_week = pd.get_dummies(aapl_diff.DAY_OF_WEEK)
day_of_week = day_of_week.rename(columns={0:'MON',1:'TUE', 2:'WED',3:'THUR',4:'FRI'})
aapl_diff = aapl_diff.join(day_of_week, how = 'outer')

In [0]:
def movement(df):
  orig_df = df.copy()
  movement= [0 if x< 0 else 1 for x in orig_df['DIFF_PRICE_OPEN']]
  movement = pd.DataFrame({'MOVEMENT': movement})
  
  return movement

    

In [0]:
def n_timesteps(df, n, feature):
  feature_df = df[[feature]]
  #temp = (feature_df - feature_df.min()) /(feature_df.max()-feature_df.min())
  temp = feature_df
  for i in range(1,n+1):
    shifted  = temp.shift(i).loc[:,feature]
    feature_df = feature_df.join(shifted, how = 'outer', rsuffix = i)

  return feature_df[n:-1]


In [0]:
n = 2
df_open = n_timesteps(aapl_diff, n,'DIFF_PRICE_OPEN')
df_close = n_timesteps(aapl_diff,n, 'DIFF_PRICE_CLOSE')
df_high = n_timesteps(aapl_diff,n, 'DIFF_PRICE_HIGH')
df_low = n_timesteps(aapl_diff,n, 'DIFF_PRICE_LOW')
df_mean = n_timesteps(aapl_diff,n, 'DIFF_PRICE_MEAN')
df_vol = n_timesteps(aapl_diff,n, 'DIFF_VOLUME_MEAN')
df_open_perc = n_timesteps(aapl_diff, n, 'PERC_PRICE_OPEN')
df_close_perc = n_timesteps(aapl_diff, n, 'PERC_PRICE_CLOSE')
df_morning = n_timesteps(aapl_diff,n, 'MORNING')
df_afternoon = n_timesteps(aapl_diff, n, 'AFTERNOON')
df_evening = n_timesteps(aapl_diff, n, 'EVENING')
df_mon = n_timesteps(aapl_diff,n, 'MON')
df_tue = n_timesteps(aapl_diff,n, 'TUE')
df_wed = n_timesteps(aapl_diff,n, 'WED')
df_thur = n_timesteps(aapl_diff,n, 'THUR')
df_fri = n_timesteps(aapl_diff,n, 'FRI')
df_range = n_timesteps(aapl_diff, n,'RANGE')
df_month = n_timesteps(aapl_diff, n, 'MONTH')
df_day = n_timesteps(aapl_diff,n,'DAY')
aapl_diff.head(5)

,Unnamed: 0,PRICE_MEAN,VOLUME_MEAN,PRICE_HIGH,PRICE_LOW,PRICE_OPEN,PRICE_CLOSE,DAY,MONTH,YEAR,HOUR,MINUTE,DIFF_PRICE_OPEN,DIFF_PRICE_CLOSE,DIFF_PRICE_HIGH,DIFF_PRICE_LOW,DIFF_PRICE_MEAN,DIFF_VOLUME_MEAN,PERC_PRICE_OPEN,PERC_PRICE_CLOSE,RANGE,TIME,MORNING,AFTERNOON,EVENING,DAY_OF_WEEK,MON,TUE,WED,THUR,FRI
0,1/2/2001 9:30,14.930692,565.354331,15.0000,14.8750,14.8750,14.8750,2,1,2001,9,30,0.0000,0.0625,0.0625,-0.1875,0.006300,142.437877,0.000000,0.004202,0.2500,1,1,0,0,1,0,1,0,0,0
1,1/2/2001 9:35,14.936993,707.792208,15.0625,14.6875,14.8750,14.9375,2,1,2001,9,35,0.0625,0.1250,0.0000,0.2500,0.084389,237.147063,0.004202,0.008368,-0.2500,1,1,0,0,1,0,1,0,0,0
2,1/2/2001 9:40,15.021382,944.939271,15.0625,14.9375,14.9375,15.0625,2,1,2001,9,40,0.1250,0.0000,0.1875,0.0625,0.092909,-209.733279,0.008368,0.000000,0.1250,1,1,0,0,1,0,1,0,0,0
3,1/2/2001 9:45,15.114291,735.205992,15.2500,15.0000,15.0625,15.0625,2,1,2001,9,45,0.0000,-0.1250,-0.1875,-0.1875,-0.175774,340.533652,0.000000,-0.008299,0.0000,1,1,0,0,1,0,1,0,0,0
4,1/2/2001 9:50,14.938517,1075.739645,15.0625,14.8125,15.0625,14.9375,2,1,2001,9,50,-0.1875,-0.1250,-0.1250,-0.0625,-0.104371,-599.592856,-0.012448,-0.008368,-0.0625,1,1,0,0,1,0,1,0,0,0


In [0]:
df_open

,DIFF_PRICE_OPEN,DIFF_PRICE_OPEN1,DIFF_PRICE_OPEN2
2,0.12500,0.06250,0.00000
3,0.00000,0.12500,0.06250
4,-0.18750,0.00000,0.12500
5,-0.12500,-0.18750,0.00000
6,0.06250,-0.12500,-0.18750
7,-0.06250,0.06250,-0.12500
8,0.03125,-0.06250,0.06250
9,0.03125,0.03125,-0.06250
10,0.00000,0.03125,0.03125
11,0.06250,0.00000,0.03125


In [0]:
#This cell converts all DataFrames with n lags into numpy arrays.


array_open = df_open.iloc[:, 1:].to_numpy()
array_close = df_close.iloc[:, 1:].to_numpy()
array_high = df_high.iloc[:, 1:].to_numpy()
array_low =df_low.iloc[:, 1:].to_numpy()
array_mean = df_mean.iloc[:, 1:].to_numpy()
array_vol = df_vol.iloc[:, 1:].to_numpy()
array_open_perc = df_open_perc.iloc[:, 1:].to_numpy()
array_close_perc = df_close_perc.iloc[:, 1:].to_numpy()
array_morning = df_morning.iloc[:, 1:].to_numpy()
array_afternoon =df_afternoon.iloc[:, 1:].to_numpy()
array_evening = df_evening.iloc[:, 1:].to_numpy()
array_mon = df_mon.iloc[:, 1:].to_numpy()
array_tue = df_tue.iloc[:, 1:].to_numpy()
array_wed = df_wed.iloc[:, 1:].to_numpy()
array_thur = df_thur.iloc[:, 1:].to_numpy()
array_fri = df_fri.iloc[:, 1:].to_numpy()
array_range = df_range.iloc[:, 1:].to_numpy()
array_month = df_month.iloc[:, 1:].to_numpy()
array_day = df_day.iloc[:, 1:].to_numpy()

In [0]:
x_val = np.stack((array_open, array_close,array_high, array_low, array_mean, array_vol, array_open_perc, array_close_perc, array_mon,array_tue,array_wed,array_thur,array_fri,array_morning,array_afternoon, array_evening, array_range, array_month,array_day), axis=1)

In [0]:
y_val = movement(df_open[['DIFF_PRICE_OPEN']]).to_numpy()
y_val[:10]

array([[1],
       [1],
       [0],
       [0],
       [1],
       [0],
       [1],
       [1],
       [1],
       [1]])

In [0]:
array_open = 0
array_close = 0
array_high = 0
array_low =0
array_mean = 0
array_vol = 0
array_open_perc = 0
array_close_perc = 0
array_time_of_day = 0
array_day_of_week = 0
array_range =0
array_month = 0
array_day = 0

df_open = 0
df_close = 0
df_high = 0
df_low = 0
df_mean = 0
df_vol =0
df_open_perc = 0
df_close_perc = 0
df_time_of_day =0
df_day_of_week = 0
df_range = 0
df_month = 0
df_day = 0

In [0]:
gc.collect()

214

In [0]:
"" #This cell converts the non-lagged data into training array and test array.

#y_val = aapl_diff[['MOVEMENT']].to_numpy()[1:]
# x_val = aapl_diff[['MONTH','DAY','DAY_OF_WEEK','TIME','PERC_PRICE_OPEN','PERC_PRICE_CLOSE','DIFF_VOLUME_MEAN','DIFF_PRICE_MEAN','DIFF_PRICE_HIGH','DIFF_PRICE_LOW','RANGE']]
# x_val = (x_val- x_val.min())/(x_val.max()-x_val.min())
# x_val = x_val.to_numpy()[:-1]
# samples = list()
# for i in range(0,x_val.shape[0]-20000):
# 	# grab from i to i + 200
# 	sample = x_val[i:i+20000]
# 	samples.append(sample)
# samples = np.array(samples)
# len(samples)

''

# Differencing per Day


In [0]:

# movement_lag_data = n_lag_steps_diff(aapl, 80)
# movement_onlylags = movement_lag_data.copy().drop(columns = 'PRICE')

# movement_onlyprice = (movement_lag_data[['PRICE']])
# movement_onlylags = movement_onlylags.to_numpy()
# movement_onlyprice = movement_onlyprice.to_numpy()
# print(movement_onlylags.shape[0])
# movement_trainX = movement_onlylags[:round(len(movement_lag_data)*.8)]
# movement_trainY = movement_onlyprice[:round(len(movement_onlyprice)*.8)]
# movement_testX = movement_onlylags[round(len(movement_lag_data)*.8):]
# movement_testY = movement_onlyprice[round(len(movement_onlyprice)*.8):]
# movement_onlylags.reshape(movement_onlylags.shape[0], movement_onlylags.shape[1], 1)
# print(movement_onlylags.shape)
# movement_onlylags.reshape(movement_onlylags.shape[0],1, movement_onlylags.shape[1])
# movement_onlylags.shape[0]

# Differencing per Min


In [0]:
# movement_lags_min = movement(aapl_min)
# movement_lag_data_min = n_lag_steps_diff(aapl_min, 10)
# movement_onlylags_min = movement_lag_data_min[['LAG1','LAG2','LAG3','LAG4','LAG5','LAG6','LAG7','LAG8','LAG9','LAG10']]
# movement_onlyprice_min = movement_lag_data_min[['PRICE']]
# movement_onlylags_min= movement_onlylags_min.to_numpy()
# movement_onlyprice_min = movement_onlyprice_min.to_numpy()
# movement_trainX_min = movement_onlylags_min[:round(len(movement_lag_data_min)*.8)]
# movement_trainY_min = movement_onlyprice_min[:round(len(movement_onlyprice_min)*.8)]
# movement_testX_min = movement_onlylags_min[round(len(movement_lag_data_min)*.8):]
# movement_testY_min = movement_onlyprice_min[round(len(movement_onlyprice_min)*.8):]
# movement_onlylags_min.reshape(movement_onlylags_min.shape[0], movement_onlylags_min.shape[1], 1)
# movement_lag_data_min

# CNN Architecture

In [0]:
def evaluate_model(trainX, trainy, testX, testy, batch_size, epochs,verbose, model_weights = None):
  #look into automatic keras neural network layer choice
  #SVM
    

      verbose, epochs, batch_size = verbose, epochs, batch_size
      n_timesteps, n_features, n_outputs = 80, 1, 1

      #trainX = trainX.reshape(trainX.shape[0], trainX.shape[1],1)
      #testX = testX.reshape(testX.shape[0], testX.shape[1],1)
      
      model = Sequential()
      model.add(CuDNNLSTM(40, stateful = True, return_sequences=True,batch_input_shape = (batch_size, trainX.shape[1], trainX.shape[2])))
      
      
      #model.add(Conv1D( filters = 3, kernel_size = 1, activation='relu'))
      #model.add(Reshape((1,1,80,1)))
      #model.add(TimeDistributed(AveragePooling1D()))
      #model.add(TimeDistributed(Conv1D(100, 10, activation='relu')))
      #model.add(TimeDistributed(Flatten()))
      #model.add(LSTM(60, stateful = True, return_sequences=True,batch_input_shape = (batch_size, trainX.shape[1], trainX.shape[2])))
      model.add(CuDNNLSTM(120, stateful = True, return_sequences=True))
      model.add(CuDNNLSTM(120, stateful = True, return_sequences=True))
      model.add(CuDNNLSTM(40, stateful = True))
      #model.add(Dropout(0.4))
      model.add(Dense(50, activation='relu'))
     
      model.add(Dropout(0.4))
      model.add(Dense(1, activation = 'sigmoid'))
      #opt = optimizers.adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=1, decay=0.001, amsgrad=False)

             
      if model_weights != None:

        model.load_weights("/content/drive/My Drive/Stock Price Prediction Research/weights.best.hdf5")
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
      else:
        model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
      # fit network
      filepath="/content/drive/My Drive/Stock Price Prediction Research/weights.best.hdf5"
      checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=0, save_best_only=True, mode='max')
      callbacks_list = [checkpoint]

      model.fit(trainX, trainy, epochs=epochs, batch_size=batch_size, verbose=verbose, callbacks = callbacks_list, shuffle = False)
      # evaluate model"
      _, accuracy = model.evaluate(testX, testy, batch_size=batch_size, verbose=0)
      
      pred = model.predict(testX, batch_size = batch_size, verbose = 0)
      weights = True
      clear_session()
      
      return pred, weights,accuracy

In [0]:
def walk_forward(initial_epochs, window, batch_size, wf_epochs):
  prediction_days = []
  actual_days = []
  walk_train = []
  walk_test = []

  x = x_val
  y = y_val
  # window = 2**13 *16
  # batch_size = 2 **10
  multiplier = 2 **10
  initial_mult = 2 **12
  #initial_batch_size =2 **10
  accuracy_days = pd.DataFrame(columns=['Minute','Accuracy'])
  p, weights, acc = evaluate_model(x[:window], y[:window], x[window:window+initial_mult], y[window:window+initial_mult], batch_size, initial_epochs, 2)
  print(acc)

  for i in range (1,500):
    trainx = x[i:i+window]
    trainy = y[i:i+window]
    
    
    testx = x[window + i: window + batch_size+i]
    testy = y[window + i: window + batch_size+i]

    pred, weights,acc = evaluate_model(trainx, trainy, testx, testy, batch_size, wf_epochs,2, weights)
    
    if pred[0][0] > .5:
      prediction_days.extend([1])
    else:
      prediction_days.extend([0])
  
    actual_days.extend(testy[0])
  
    # day = pd.DataFrame({'Minute': i*5 +window, 'Accuracy': [acc]})
    # accuracy_days = accuracy_days.append(day,ignore_index=True)
    
    print("Minute"+str(i*5), acc)
    print(pred[0])
    print(testy[0])
  p_r = precision_recall_fscore_support(actual_days,prediction_days,labels = [0,1])
  return prediction_days, p_r, actual_days
    
  

In [0]:

def plot_prediction(actual_days, prediction_days, total, cutoff):
  binary_predict = np.array([1 if x> .5 else -1 for x in actual_days])
  binary_actual = np.array([1 if x==1 else -1 for x in prediction_days])
  #print(binary_actual, prediction_days)
  tot_predict = 0
  tot_actual =0 
  plot_predict = []
  plot_actual =[]
  
  for i in range((len(binary_predict))):
    
    tot_predict += binary_predict[i]
    tot_actual += binary_actual[i]
    plot_predict.append(tot_predict)
    plot_actual.append(tot_actual)
  plot_df = (pd.DataFrame({'Predict' : plot_predict, "Actual" : plot_actual}))

  plot_df.plot(figsize = (10,10))
  plt.legend()
  plt.axvline(x=cutoff)

  plt.show()

In [0]:
#This cell is meant to tune the hyper parameters for the RNN.

epoch_range = [100, 200, 300, 400, 500]
training_window_size = [2**13 *11, 2**13 *12, 2**13 *13, 2**13 *14, 2**13 *15]
wf_epoch_range = [1,3,5,10,20]
batch_size = [2**9,2**10, 2**11, 2**12]
tuning = pd.DataFrame(columns = ['Epoch Size','Training Set Size','WF Epoch Size','Batch Size', 'Precision Down','Precision Up', 'Recall Down', 'Recall Up'])

for i in epoch_range:
  for j in training_window_size:
    for k in wf_epoch_range:
        for b in batch_size:
          prediction, prec_recall, actual_days = walk_forward(i, j, b, k)
          
          p_down = prec_recall[0][0]
          p_up = prec_recall[0][1]
          r_down = prec_recall[1][0]
          r_up = prec_recall[1][1]
          plot_prediction(actual_days, prediction, 0,0)
          row = {'Epoch Size': i,'Training Set Size':j,'WF Epoch Size':k,'Batch Size':b, 'Precision Down':p_down,'Precision Up':p_up, 'Recall Down':r_down, 'Recall Up':r_up}
          tuning = tuning.append(row, ignore_index = True)
tuning          





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/100


InvalidArgumentError: ignored

In [0]:
plot_prediction(actual_days, prediction_days, 0,0)

In [0]:
def profit(predicted, prices, lags, capital, initial_window, multiplier):
  figure(num=None, figsize=(25, 10), dpi=80, facecolor='w', edgecolor='k')
  money = []
  shares = 0
  last_price = 0
  num_predicted = len(predicted)
  sell = initial_window
  prices = prices[initial_window+ multiplier : initial_window +multiplier + num_predicted]

  for pred, price in zip(predicted, prices):
    num_of_shares = 1

    if pred == 0:
      if shares - num_of_shares <= 0:
         money.append(shares * price + capital)
         sell +=1
         continue
      capital += price *num_of_shares
      shares -= num_of_shares
      print('Sold at '+ str(price))
      plt.axvline(sell)
    else:
      
      if capital < price:
        money.append(shares * price + capital)
        sell +=1
        continue
      plt.axvline(sell, color = 'g')
      capital -= price* num_of_shares
      shares += num_of_shares
      print('Bought at '+ str(price))
    money.append(shares * price + capital)  
    last_price = price
    sell +=1
  
  
  graph= pd.DataFrame({'Total': money,'Price': prices})  
  
  ax1 = graph.Total.plot(figsize = (100,100))
  
  ax2 = ax1.twinx()
  ax2.spines['right'].set_position(('axes', 1.0))
  graph.Price.plot(color = 'r', secondary_y = True)
  plt.legend()
  plt.show()
  return shares * last_price + capital


In [0]:

profit(prediction_days, aapl['PRICE'], 80, 3000,2000, 10)

In [0]:
aapl.iloc[1040:1110]

In [0]:
 
tot_predict = 0
tot_actual = 0

binary_predict = np.array([1 if x[0]> .5 else -1 for x in y_output])
binary_actual = np.array([1 if x[0]==1 else -1 for x in movement_testY])
plot_predict = []
plot_actual =[]
for i in range(len(binary_predict)):
  tot_predict += binary_predict[i]
  tot_actual += binary_actual[i]
  plot_predict.append(tot_predict)
  plot_actual.append(tot_actual)
plot_df = (pd.DataFrame({'Predict' : plot_predict, "Actual" : plot_actual}))
plot_df.plot()
plt.legend()
plt.show()
precision_recall_fscore_support(binary_predict, binary_actual)

In [0]:
y_output_min = evaluate_model(movement_trainX_min, movement_trainY_min, movement_testX_min, movement_testY_min, 5000, 200)

In [0]:
 
tot_predict = 0
tot_actual = 0

binary_predict = np.array([1 if x[0]> .5 else 0 for x in y_output_min])
binary_actual = np.array([1 if x[0]==1 else 0 for x in movement_testY_min])
plot_predict = []
plot_actual =[]
for i in range(len(binary_predict)):
  tot_predict += binary_predict[i]
  tot_actual += binary_actual[i]
  plot_predict.append(tot_predict)
  plot_actual.append(tot_actual)
plot_df = (pd.DataFrame({'Predict' : plot_predict, "Actual" : plot_actual}))
plot_df.plot()
plt.legend()
plt.show()



In [0]:
precision_recall_fscore_support(binary_predict, binary_actual)


In [0]:
movement_testY